In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import glob
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import fuzz
import re
import pyodbc
import sqlalchemy

### set the filepath and the wildcard mask to read all excel files

In [2]:
# inpath = r"C:\Users\A186999\OneDrive - Standard Bank\Pyfun\files"
inpath = "C:\\Users\\test\\pyfortress\\pcard\\infiles"
outpath = "C:\\Users\\test\\pyfortress\\pcard\\outfiles"

filenames = glob.glob(f"{inpath}\*.xlsx")

#### Create a Function to Load files which takes a list parameter (The directory where new files are dropped daily/monthly)

#### Initialize an Empty Dataframe that would have the data from all the excel files. 
##### A Column list is also created to make the different column names uniform during merge

#### Using a for-loop, we loop through the directory, reading an excel file, rename the column names for consistency and appending it to the CombinedFrame DF created earlier. 
#### We also know from trend that any line without a batch-id is for calculations/subtotals only. We would go ahead and remove those immediately as we load the files

In [3]:
def MergeFilesInitialLoad(filenames):
    combinedFrame = pd.DataFrame()
    col = ['division',
    'batch_id',
    'tran_date',
    'pstd_date',
    'merchant_name',
    'tran_amt',
    'tran_crncy',
    'orig_amt',
    'orig_crcny',
    'gl_acct',
    'gl_acct_desc',
    'cc_wbs_ord',
    'cc_wbs_ord_desc',
    'merchant_type',
    'merchant_type_desc',
    'purpose']

    for file in filenames:
        df =pd.read_excel(file, sheet_name=False)
        df.columns = col
        # df['filename'] = file
        df = df[~df['batch_id'].isnull()]
    
        combinedFrame = combinedFrame.append(df, ignore_index=True)

    return combinedFrame

In [4]:
combinedFrame = MergeFilesInitialLoad(filenames)

In [5]:
# combinedFrame.to_csv(".\workframereload.csv", index=False)
# combinedFrame.info()

#### We Copy the CombinedFrame into a WorkFrame we can clean further.

In [6]:
workFrame = combinedFrame

### Dealing with Missing Data

In [7]:
## We update Missing "Division Names" with "Unknown" since there's no place else we can infer this data from. 
workFrame[['division']] = workFrame[['division']].fillna(value='UNKNOWN')

## We update missing 'dates' with the value of the previous date, since the files were loaded sequentially. 
workFrame['tran_date'] = workFrame['tran_date'].ffill()

##Batch_id being the primary key, we would also drop duplicates, keeping the last value
workFrame = workFrame.drop_duplicates('batch_id', keep='last')


#### The Merchant Type Code is a 4 digit integer code. Hence we convert the column to Integer
workFrame["merchant_type"] = workFrame["merchant_type"].astype('Int64')

# workFrame[workFrame['tran_date'].isna()]
workFrame.isnull().sum()
# workFrame.info()
# workFrame[workFrame['division'].isnull()]

division                 0
batch_id                 0
tran_date                0
pstd_date                0
merchant_name            0
tran_amt                 0
tran_crncy               0
orig_amt                 0
orig_crcny               0
gl_acct                  0
gl_acct_desc             0
cc_wbs_ord             645
cc_wbs_ord_desc        588
merchant_type           64
merchant_type_desc       0
purpose               1822
dtype: int64

## Cleaning and preparing Merchant Details
#### We Commence data cleaning on the merchant related fields

In [8]:
## We will attempt to clean up the merchant name a bit using regex and fuzzyWuzzy. we would copy the merchant_name to another column (raw_merchant_name) clean and replace the name.. 
## Because of how dirty the merchant name if the business needs it, it can be further modeled and improved. Due to time constraints...
# I have precleaned the names manually, using the precleaned data, I would use the fuzzyWuzzy model to find similarities and update accordingly

workFrame['raw_merchant_name'] = workFrame['merchant_name']

raw_merchant_list = list(workFrame.merchant_name.unique())

cleaned_merchant_list = pd.read_csv('./cleanedMerchantList.csv')#, header = None, index_col = 0, squeeze = True)
cleaned_merchant_list = list(cleaned_merchant_list['0'])

pattern = r'((#\d{1,}\s)|(\s# \d{1,}.+)|(\s#\d{1,}.+)|( # 1| #1)|(\s#\d))'
raw_merchant_list =[re.sub(pattern, '', w) for w in raw_merchant_list]

# def match_names(name, list_names, min_score=0):
#     max_score = -1
#     max_name = ''
#     for x in list_names:
#         score = fuzz.ratio(name, x)
#         if (score > min_score) & (score > max_score):
#             max_name = x
#             max_score = score
#     return (max_name, max_score)

# names = []
# for x in raw_merchant_list:
#     match = match_names(x, cleaned_merchant_list, 0)
#     if match[1] >= 60:
#         name = ('(' + str(x), str(match[0]) + ')')
#         names.append(name)
#     else:
#         name = ('(' + str(x), str(x)+ ')')
#         names.append(name)
# name_dict = dict(names)

# #Using the dictionary to replace the keys with the values in the 'name' column for the second dataframe
# workFrame.merchant_name = workFrame.merchant_name.replace(name_dict)

In [9]:
cleaned_merchant_list

["MARK'S",
 "MARK'S AND SPENCER",
 'SPORT CHEK',
 'STEREO & VIDEO',
 'MYROUTEON',
 '4IMPRINT',
 'A BUCK OR TWO',
 'A TOWING SERVICE',
 'A.B.C. RECREATION LTD',
 'AIRLINE TAXI AND V',
 'AAROPORT LIMOUSINE SERVICE',
 'ABLE LOCK SERVICE LTD',
 'ABOVE GROUND ENTERPRISE',
 'ABSOLUTE DOLLAR',
 'ACAPULCO POOLS',
 'ACE AWARDS INC',
 'ACKLANDS GRAINGER INC',
 'ACRYLICS BY DESIGN INC',
 'ACT PRO ONT',
 'ACTION CAR & TRUCK',
 'ACTION HYDRAULICS LTD',
 'ACTION TROPHY & ENGRAV',
 'ACTIVITY CONNECTIO CO',
 'ADI  T1',
 'ADOBE',
 'ADONIS SCARBOROUGH',
 'ADRIATIC GLASS & MIRROR',
 'ADVANTAGE PACKAGING',
 'ADWEAR',
 'AEROFLEET SERVICES',
 'AFFINITY ELECTRONICS',
 'AFFORDABLE TEXTILES',
 'AGINCOURT TROPHY & GIF',
 "KATHY'S CATERING",
 'AIR CAN',
 'AIRLINE LIMOUSINE SERVICES',
 'AL PREMIUM FOOD MART',
 'ALCONA HOME HARDWARE',
 'ALEX DRY CLEANING',
 'ALGOMA ORCHARDS',
 'ALGOOD CASTERS LIMITED',
 'ALLIANCE PROMOTIONS',
 'ALPHA CROWD CONTROL',
 'ALPINE LAWN & GARDEN',
 'ALUMNI WESTER',
 'AMACOR MARKETING INC

In [10]:
# we will separate the dataframe by the merchant type description. work on each of the dataframes and merge later
newMcc_df = workFrame[workFrame['merchant_type_desc']=="NEW MCC CODE"]
withoutNewMcc_df =  workFrame[workFrame['merchant_type_desc']!="NEW MCC CODE"]



In [11]:
## Creating a unique Dataframe of merchant_type_descriptions. We would use this eventually to create and populate our Merchant's table but more importantly to retrieve and fill the missing merchant_type(mcc) codes

distinct_mcc = withoutNewMcc_df['merchant_type_desc'].unique()
df_distinct_mcc= pd.DataFrame(distinct_mcc)
df_distinct_mcc.columns = ["merchant_type_desc"]

# df_distinct_mcc.info()

In [12]:
## Here we create a lookup frame/dictionary we would use to retrieve and match the  missing mcc codes

df_mccframe = withoutNewMcc_df[['merchant_type_desc', 'merchant_type']]

df_mccframe_dict = dict(df_mccframe.values)

In [13]:
## Create a new column of merchant_type using the dictionary to retrieve the mcc codes. We would also use the index function to create a merchant_id in line with our data model for the Merchant Table. 
## The last_index variable is used to store the value of the last index. We would keep this handy for the other part of the df we separated "newMcc_df"

df_distinct_mcc['merchant_type'] = df_distinct_mcc.merchant_type_desc.map(df_mccframe_dict)
# df_distinct_mcc['merchant_id'] = df_distinct_mcc.index
# last_index = len(df_distinct_mcc)


In [14]:
## using this unique dataframe of mcc codes and description we merge it with the "withoutNewMcc_df" hence, creating a new merchant_type column (merchant_type_y) with no missing code

withoutNewMcc_df = pd.merge(withoutNewMcc_df, df_distinct_mcc, on='merchant_type_desc')

In [15]:
## Copy and replace the old merchant_type_x column with the new merchant_type_y column and rename the column accordingly. 
withoutNewMcc_df['merchant_type_x'] = withoutNewMcc_df['merchant_type_y']
withoutNewMcc_df.rename(columns={'merchant_type_x':'merchant_type'},inplace=True)

In [16]:
# withoutNewMcc_df.isnull().sum()

#### We repeat same for the second part of the dataframe ("newMcc_df") with the "New MCC Codes" but in this case we have the merchant type codes and description. We just want to get the unique merchant_id for each unique merchant_type code. 

In [17]:
distinct_wmcc = newMcc_df['merchant_type'].unique()
df_distinct_wmcc= pd.DataFrame(distinct_wmcc)
df_distinct_wmcc.columns = ["merchant_type"]
df_distinct_wmcc['merchant_type_desc'] = "NEW MCC CODE"
# df_distinct_wmcc



In [18]:
#Since the goal is to later merge this dataframe to the first df - withoutNewMcc_df, we want to keep the shape and columns all aligned. we are also modifying the index(merchant_id) to continue from the last index in the first df

# df_distinct_wmcc.index = np.arange(last_index, last_index+len(df_distinct_wmcc))
# df_distinct_wmcc['merchant_id'] = df_distinct_wmcc.index

In [19]:
newMcc_df = pd.merge(newMcc_df, df_distinct_wmcc, on='merchant_type')
newMcc_df.rename(columns={'merchant_type_desc_x':'merchant_type_desc'},inplace=True)

In [25]:
merchantWorkFrame_Df = pd.concat([withoutNewMcc_df, newMcc_df])
# df_distinct_wmcc.info()
# merchantWorkFrame_df.info()
# merchantWorkFrame_df.isnull().sum()

In [21]:
## We merge these two unique Merchant DF (it contains Unique Merchant Types and their descriptions) We will use to merge with a 
#unique merchant name list in order to get the unique merchant details for our db

# uniquemerchantdf = pd.concat([df_distinct_wmcc, df_distinct_mcc])
# uniquemerchantdf.info()


In [22]:
# merchantWorkFrame_df.isnull().sum()

In [27]:
## We merge these two unique Merchant DF (it contains Unique Merchant Types and their descriptions) We will use to merge with a 
#unique merchant name list in order to get the unique merchant details for our db
# 
uniquemerchantdf = pd.concat([df_distinct_wmcc, df_distinct_mcc])

distinct_mname = merchantWorkFrame_Df['merchant_name'].unique()
df_distinct_mname= pd.DataFrame(distinct_mname)
df_distinct_mname.columns = ["merchant_name"]
df_mnameframe = merchantWorkFrame_Df[['merchant_name', 'merchant_type']]
df_mnameframe_dict = dict(df_mnameframe.values)
df_distinct_mname['merchant_type'] = df_distinct_mname.merchant_name.map(df_mnameframe_dict)

## and here we merge the tables

merchantsTableDf = pd.merge(df_distinct_mname, uniquemerchantdf, on='merchant_type')
merchantsTableDf = merchantsTableDf.drop_duplicates(subset=['merchant_name'], keep='last')

In [28]:
merchantsTableDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21741 entries, 0 to 21772
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   merchant_name       21741 non-null  object
 1   merchant_type       21741 non-null  int64 
 2   merchant_type_desc  21741 non-null  object
dtypes: int64(1), object(2)
memory usage: 679.4+ KB


## Cleaning and Prepping Cost Center/WBS Orders
#### We Commence data cleaning on the CostCenter/WBS related fields

In [30]:
ccWorkFramedf = merchantWorkFrame_Df

# ccWorkFramedf.info()

In [31]:
# We find and replace invalid characters '-' and NaN on the cc_wbs_ord column, using '000001' to represent such cost centers

ccWorkFramedf.loc[ccWorkFramedf['cc_wbs_ord'] == '-', ['cc_wbs_ord']] = '000001'
ccWorkFramedf.loc[ccWorkFramedf['cc_wbs_ord'].isnull(), ['cc_wbs_ord']] = '000001'

distinctCcWbs = ccWorkFramedf['cc_wbs_ord'].unique()
df_distinctCcWbs = pd.DataFrame(distinctCcWbs)
df_distinctCcWbs.columns = ["cc_wbs_ord"]

In [32]:
## Here we create a lookup frame/dictionary we would use to retrieve and match the  missing mcc codes

df_CcWbs = ccWorkFramedf[['cc_wbs_ord', 'cc_wbs_ord_desc']]
df_CcWbs_dict = dict(df_CcWbs.values)


In [33]:
# As usual we loop through the dictionary to map cost center order description(cc_wbs_ord_desc) to the cc order no (cc_wbs_ord)
# We find and replace invalid characters '-' and NaN on the cc_wbs_ord column, using '000001' to represent such cost centers

df_distinctCcWbs['cc_wbs_ord_desc'] = df_distinctCcWbs.cc_wbs_ord.map(df_CcWbs_dict)
df_distinctCcWbs[['cc_wbs_ord_desc']] = df_distinctCcWbs[['cc_wbs_ord_desc']].fillna(value='NO DESC')

In [34]:
# ccWorkFramedf.loc[ccWorkFramedf['cc_wbs_ord'] == '-', ['cc_wbs_ord']] = '000001'

# ccWorkFramedf.loc[ccWorkFramedf['cc_wbs_ord'].isnull(), ['cc_wbs_ord']]

# ccWorkFramedf.isnull().sum()

In [35]:
# We also use the index function to create a cc_wbs_ord_id in line with our data model for the Cost Center Table. 
# df_distinctCcWbs['cc_wbs_ord_id'] = df_distinctCcWbs.index

In [36]:
ccWorkFramedf = pd.merge(ccWorkFramedf, df_distinctCcWbs, on='cc_wbs_ord')
ccWorkFramedf['cc_wbs_ord_desc_x'] = ccWorkFramedf['cc_wbs_ord_desc_y']
ccWorkFramedf.rename(columns={'cc_wbs_ord_desc_x':'cc_wbs_ord_desc'},inplace=True)

In [37]:
ccWorkFramedf.isnull().sum()

division                     0
batch_id                     0
tran_date                    0
pstd_date                    0
merchant_name                0
tran_amt                     0
tran_crncy                   0
orig_amt                     0
orig_crcny                   0
gl_acct                      0
gl_acct_desc                 0
cc_wbs_ord                   0
cc_wbs_ord_desc              0
merchant_type                0
merchant_type_desc           0
purpose                   1822
raw_merchant_name            0
merchant_type_y             86
merchant_type_desc_y    248781
cc_wbs_ord_desc_y            0
dtype: int64

## Generating Unique Divisions and IDs

In [38]:
#Make a copy of the last Comprehensive DF and copy it to division DF

divDf = ccWorkFramedf

In [39]:
distinct_div = divDf['division'].unique()
df_distinct_div= pd.DataFrame(distinct_div)
df_distinct_div.columns = ["division"]
df_distinct_div['division_id'] = df_distinct_div.index
divWorkFramedf = pd.merge(divDf, df_distinct_div, on='division')


In [40]:
df_distinct_div['division_id'] = df_distinct_div.index
divWorkFramedf = pd.merge(divDf, df_distinct_div, on='division')

In [41]:
# divWorkFramedf.info()
divWorkFramedf.to_csv(f"{outpath}\\divWorkFramedf.csv", index=False)

In [42]:
##Extract the columns we need according to our Model in prep. of Writing to DB

transactionDf = divWorkFramedf


In [43]:
transactionTableDf = transactionDf[["batch_id", "division", "tran_date", "pstd_date", "merchant_name", "tran_amt", "tran_crncy", "orig_amt", "orig_crcny", "gl_acct", "cc_wbs_ord", "purpose"]]
# transactionTableDf.to_csv(r"C:\Users\A186999\OneDrive - Standard Bank\pcard\outfiles\transactionTableDf.csv", index=False)
# transactionTableDf.to_sql(con=dbEngine, schema='dbo', name='purchases', if_exists="append", index=False, chunksize=1000)


In [44]:
# merchantTableDf = pd.concat([df_distinct_mcc, df_distinct_wmcc])
merchantsTableDf.to_csv(f"{outpath}\\merchantTableDf.csv", index=False)


In [45]:
costCenterTable = df_distinctCcWbs

costCenterTable.isnull().sum()
# df_distinctCcWbs[df_distinctCcWbs['cc_wbs_ord'].isnull()]

cc_wbs_ord         0
cc_wbs_ord_desc    0
dtype: int64

In [46]:
## Creating a our gl df. We would use this eventually to create and populate our General Ledger's table 

distinct_glc = divWorkFramedf['gl_acct'].unique()
df_distinct_glc= pd.DataFrame(distinct_glc)
df_distinct_glc.columns = ["gl_acct"]
df_glc = ccWorkFramedf[['gl_acct', 'gl_acct_desc']]
df_glc_dict = dict(df_glc.values)
df_distinct_glc['gl_acct_desc'] = df_distinct_glc.gl_acct.map(df_glc_dict)

generalLedgerTableDf = df_distinct_glc

In [47]:
server = 'localhost\sqlexpress' 
database = 'pcard' 
username = 'ludwig' 
password = 'ludwig' 

In [48]:
constring = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
dbEngine = sqlalchemy.create_engine(constring, fast_executemany=True, connect_args={'connect_timeout': 10}, echo=False)

In [49]:
try:
    with dbEngine.connect() as con:
        con.execute("SELECT 1")
    print('DB Engine and Connection is Valid')
except Exception as e:
    print(f'DB Engine is Invalid: {str(e)}')


# con.close()

DB Engine and Connection is Valid


In [50]:
try: 
    transactionTableDf.to_sql(con=dbEngine, schema='dbo', name='purchases', if_exists="append", index=False, chunksize=1000)
except pyodbc.ProgrammingError as e:
    print(f"Write Error: {str(e)}")

print("continued")


IntegrityError: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__purchase__DBFC0431DBDEDB8A'. Cannot insert duplicate key in object 'dbo.purchases'. The duplicate key value is (4608-1). (2627) (SQLExecute); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__purchase__DBFC0431DBDEDB8A'. Cannot insert duplicate key in object 'dbo.purchases'. The duplicate key value is (4617-1). (2627); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__purchase__DBFC0431DBDEDB8A'. Cannot insert duplicate key in object 'dbo.purchases'. The duplicate key value is (4965-479). (2627); [23000] [Microsoft][ODBC Dri